# Tech Challenge
## Análise Exploratória PNAD Covid19

## ETAPA 1 — PREPARAÇÃO INICIAL DOS DADOS

**Objetivo**:

Compreender a estrutura do conjunto de dados, identificar as colunas  disponíveis, tipos de dados, valores nulos e
possíveis inconsistências antes de iniciar as análises.

Esta etapa é fundamental para garantir que as agregações
e análises posteriores sejam realizadas sobre dados confiáveis.


####  Importar Bibliotecas


 PySpark: manipulação e agregação de grandes volumes de dados.

 Pandas: apoio à análise e conversão de dados para visualização.

 Matplotlib: criação de gráficos para análise dos resultados.

 Boto3: integração com a AWS para envio de arquivos ao S3.

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col,
    count,
    sum,
    when,
    avg
    concat_ws
)


import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import boto3



Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Trying to create a Glue session for the kernel.
Session Type: glueetl
Session ID: e734706d-eacb-412d-8a4f-2a4c198132bd
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
Waiting for session e734706d-eacb-412d-8a4f-2a4c198132bd to get into ready status...
Session e734706d-eacb-412d-8a4f-2a4c198132bd has been created.
SyntaxError: invalid syntax (<stdin>, line 8)


In [ ]:
from pyspark.sql.functions import (
    col,
    count,
    sum,
    when,
    avg,
    concat_ws
)

#### Importar a base do S3


In [ ]:
fato = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("s3://lab-232736165724/data-output/fato_analitica_final/")


#### Exibir os tipos de dados e a tabela


In [ ]:
fato.printSchema()
fato.show(5, truncate=False)


root
 |-- capital: integer (nullable = true)
 |-- uf: integer (nullable = true)
 |-- cod_pergunta: string (nullable = true)
 |-- cod_resposta: integer (nullable = true)
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- idade: integer (nullable = true)
 |-- sexo: integer (nullable = true)
 |-- raca: integer (nullable = true)
 |-- escolaridade: integer (nullable = true)
 |-- descricao_pergunta: string (nullable = true)
 |-- descricao_resposta: string (nullable = true)
 |-- nome_uf: string (nullable = true)
 |-- descricao_capital: string (nullable = true)
 |-- cod_sexo: integer (nullable = true)
 |-- descricao_sexo: string (nullable = true)
 |-- cod_raca: integer (nullable = true)
 |-- descricao_raca: string (nullable = true)
 |-- cod_escolaridade: integer (nullable = true)
 |-- descricao_escolaridade: string (nullable = true)

+-------+---+------------+------------+----+---+-----+----+----+------------+-------------------------------------+------------------+--

## ETAPA 2 — CRIAÇÃO DE VARIÁVEIS DERIVADAS
**Objetivo**:

Criar classificações que facilitem a análise demográfica, eduzindo a granularidade da idade em faixas etárias mais adequadas para análises populacionais.

In [ ]:
# Incluir coluna região

fato = fato.withColumn(
    "regiao",
    when(col("nome_uf").isin(
        "Rondônia", "Acre", "Amazonas", "Roraima",
        "Pará", "Amapá", "Tocantins"
    ), "Norte")
    .when(col("nome_uf").isin(
        "Maranhão", "Piauí", "Ceará", "Rio Grande do Norte",
        "Paraíba", "Pernambuco", "Alagoas", "Sergipe", "Bahia"
    ), "Nordeste")
    .when(col("nome_uf").isin(
        "Minas Gerais", "Espírito Santo",
        "Rio de Janeiro", "São Paulo"
    ), "Sudeste")
    .when(col("nome_uf").isin(
        "Paraná", "Santa Catarina", "Rio Grande do Sul"
    ), "Sul")
    .when(col("nome_uf").isin(
        "Mato Grosso do Sul", "Mato Grosso",
        "Goiás", "Distrito Federal"
    ), "Centro-Oeste")
    .otherwise("Ignorado")
)


In [ ]:
# Incluir coluna faixa etaria
fato = fato.withColumn(
    "faixa_etaria",
    when((col("idade") >= 0) & (col("idade") <= 10), "0-10")
    .when((col("idade") >= 11) & (col("idade") <= 20), "11-20")
    .when((col("idade") >= 21) & (col("idade") <= 30), "21-30")
    .when((col("idade") >= 31) & (col("idade") <= 40), "31-40")
    .when((col("idade") >= 41) & (col("idade") <= 50), "41-50")
    .when((col("idade") >= 51) & (col("idade") <= 60), "51-60")
    .when((col("idade") >= 61) & (col("idade") <= 70), "61-70")
    .when(col("idade") >= 71, "71+")
    .otherwise("Ignorado")
)

In [ ]:
# Incluir coluna tipo de sintoma
fato = fato.withColumn(
    "tipo_sintoma",
    when(col("cod_pergunta").isin("B0014", "B0016"), "Respiratório grave")
    .when(col("cod_pergunta").isin("B0012", "B0013"), "Respiratório leve")
    .when(col("cod_pergunta").isin("B0011", "B0015", "B0017", "B0019"), "Sintoma geral")
)


## ETAPA 3 — ANÁLISES EXPLORATÓRIAS

#### Filtrar respostas nulas


In [ ]:
fato = fato.filter(col("descricao_resposta").isNotNull())


#### Identificar Perguntas Utilizadas na análise

In [ ]:
fato.select("cod_pergunta","descricao_pergunta") \
    .distinct() \
    .orderBy("cod_pergunta","descricao_pergunta") \
    .show(50, truncate=False)


+------------+---------------------------------------------------------------------------------------------------+
|cod_pergunta|descricao_pergunta                                                                                 |
+------------+---------------------------------------------------------------------------------------------------+
|B0011       |Na semana passada teve febre?                                                                      |
|B0012       |Na semana passada teve tosse?                                                                      |
|B0013       |Na semana passada teve dor de garganta?                                                            |
|B0014       |Na semana passada teve dificuldade para respirar?                                                  |
|B0015       |Na semana passada teve dor de cabeça?                                                              |
|B0016       |Na semana passada teve dor no peito?                              

#### Exibir o agrupamento das respostas
Nesta etapa são realizados agrupamentos para reduzir a granularidade dos dados,
transformando registros individuais em informações agregadas.

A pré análise é importante para identificar quais agregações fazem sentido para a analise final.

Os gráficos ajudam a visualizar melhor os dados. Eles serão salvos no S3.

In [ ]:
agrupamento_Geral = (
    fato
    .groupBy(
        "descricao_pergunta",
        "descricao_resposta"
    )
    .agg(
        count("*").alias("quantidade")
    )
    .orderBy("descricao_pergunta")
)

agrupamento_Geral.show(20, truncate=False)

+--------------------------------------------------------------------------------+--------------------------------------+----------+
|descricao_pergunta                                                              |descricao_resposta                    |quantidade|
+--------------------------------------------------------------------------------+--------------------------------------+----------+
|Ao procurar o hospital, teve que ficar internado por um dia ou mais             |Não                                   |6924      |
|Ao procurar o hospital, teve que ficar internado por um dia ou mais             |Ignorado                              |36        |
|Ao procurar o hospital, teve que ficar internado por um dia ou mais             |Sim                                   |765       |
|Ao procurar o hospital, teve que ficar internado por um dia ou mais             |Não foi atendido                      |35        |
|Durante o período da pandemia alguém deste domicílio solicitou algum

In [ ]:
agrupamento_UF = (
    fato
    .groupBy(
        "nome_uf",
        "descricao_pergunta",
        "descricao_resposta"
    )
    .agg(
        count("*").alias("quantidade")
    )
    .orderBy("nome_uf", "descricao_pergunta")
)

agrupamento_UF.show(50, truncate=False)


+-------+-------------------------------------------------------------------------------------------------+--------------------------------------+----------+
|nome_uf|descricao_pergunta                                                                               |descricao_resposta                    |quantidade|
+-------+-------------------------------------------------------------------------------------------------+--------------------------------------+----------+
|Acre   |Ao procurar o hospital, teve que ficar internado por um dia ou mais                              |Não                                   |97        |
|Acre   |Ao procurar o hospital, teve que ficar internado por um dia ou mais                              |Sim                                   |5         |
|Acre   |Durante o período da pandemia alguém deste domicílio solicitou algum empréstimo?                 |Sim, e pelo menos um morador conseguiu|584       |
|Acre   |Durante o período da pandemia alguém deste 

In [ ]:
agrupamento_sexo = (
    fato
    .groupBy(
        "descricao_sexo",
        "descricao_pergunta",
        "descricao_resposta"
    )
    .agg(
        count("*").alias("quantidade")
    )
    .orderBy("descricao_sexo", "descricao_pergunta")
)

agrupamento_sexo.show(50, truncate=False)

+--------------+--------------------------------------------------------------------------------+--------------------------------------+----------+
|descricao_sexo|descricao_pergunta                                                              |descricao_resposta                    |quantidade|
+--------------+--------------------------------------------------------------------------------+--------------------------------------+----------+
|Homem         |Ao procurar o hospital, teve que ficar internado por um dia ou mais             |Ignorado                              |18        |
|Homem         |Ao procurar o hospital, teve que ficar internado por um dia ou mais             |Não                                   |2850      |
|Homem         |Ao procurar o hospital, teve que ficar internado por um dia ou mais             |Não foi atendido                      |15        |
|Homem         |Ao procurar o hospital, teve que ficar internado por um dia ou mais             |Sim            

In [ ]:
#Quantidade por UF
fato \
    .filter(col("descricao_resposta") == "Sim") \
    .groupBy("uf", "nome_uf", "descricao_pergunta") \
    .agg(count("*").alias("qtd_casos")) \
    .orderBy("uf", "qtd_casos", ascending=False) \
    .show(20, truncate=False)


+---+----------------+---------------------------------------------------------------------------------------------------+---------+
|uf |nome_uf         |descricao_pergunta                                                                                 |qtd_casos|
+---+----------------+---------------------------------------------------------------------------------------------------+---------+
|53 |Distrito Federal|Na semana passada, por pelo menos uma hora, trabalhou ou fez algum bico?                           |5309     |
|53 |Distrito Federal|Na semana passada, estava temporariamente afastado de algum trabalho?                              |671      |
|53 |Distrito Federal|Na semana passada teve dor de cabeça?                                                              |342      |
|53 |Distrito Federal|Tem mais de um trabalho                                                                            |251      |
|53 |Distrito Federal|Providência tomada  para recuperar dos sintomas

In [ ]:
# Quantidade por sexo
fato \
    .groupBy("descricao_sexo") \
    .agg(count("*").alias("total")) \
    .orderBy("total", ascending=False) \
    .show()


+--------------+-------+
|descricao_sexo|  total|
+--------------+-------+
|        Mulher|6528695|
|         Homem|5936357|
+--------------+-------+


#### Função para salvar o gráfico no formato png

In [ ]:
def salvar_no_s3(local_path, bucket, s3_path):
    s3 = boto3.client("s3")
    s3.upload_file(local_path, bucket, s3_path)


### Sintomas

#### Sintomas mais Comuns

In [ ]:
sintomas = (
    fato
    .filter(col("cod_pergunta").isin(
        "B0011","B0012","B0013","B0014",
        "B0015","B0016","B0017","B0019"
    ))
    .filter(col("descricao_resposta") == "Sim")
    .groupBy(col("descricao_pergunta").alias("Sintomas"))
    .agg(count("*").alias("qtd"))
    .orderBy("qtd", ascending=False)
)

sintomas.show(truncate=False)




+-------------------------------------------------+-----+
|Sintomas                                         |qtd  |
+-------------------------------------------------+-----+
|Na semana passada teve dor de cabeça?            |18273|
|Na semana passada teve tosse?                    |11640|
|Na semana passada teve dor de garganta?          |11222|
|Na semana passada teve dor no peito?             |6828 |
|Na semana passada teve fadiga?                   |6561 |
|Na semana passada teve dificuldade para respirar?|6429 |
|Na semana passada teve febre?                    |6114 |
|Na semana passada teve náusea?                   |4204 |
+-------------------------------------------------+-----+


In [ ]:
# Gráfico Sintomas
'''
pdf = sintomas.toPandas()

plt.figure(figsize=(10,6))
plt.barh(pdf["Sintomas"], pdf["qtd"])
plt.title("Sintomas mais comuns na população")
plt.xlabel("Quantidade de pessoas")
plt.ylabel("Sintoma")
plt.gca().invert_yaxis()

plt.tight_layout()
plt.savefig("/tmp/sintomas.png")
plt.close()

salvar_no_s3(
    "/tmp/sintomas.png",
    "lab-232736165724",
    "data-output/graficos/sintomas.png"
)

'''


'\npdf = sintomas.toPandas()\n\nplt.figure(figsize=(10,6))\nplt.barh(pdf["Sintomas"], pdf["qtd"])\nplt.title("Sintomas mais comuns na população")\nplt.xlabel("Quantidade de pessoas")\nplt.ylabel("Sintoma")\nplt.gca().invert_yaxis()\n\nplt.tight_layout()\nplt.savefig("/tmp/sintomas.png")\nplt.close()\n\nsalvar_no_s3(\n    "/tmp/sintomas.png",\n    "lab-232736165724",\n    "data-output/graficos/sintomas.png"\n)\n\n'


#### Sintomas por grupo

In [ ]:
# Sintomas por região
sintomas_regiao = (
    fato
    .filter(col("cod_pergunta").isin(
        "B0011","B0012","B0013","B0014",
        "B0015","B0016","B0017","B0019"
    ))
    .filter(col("descricao_resposta") == "Sim")
    .groupBy(col("descricao_pergunta").alias("Sintomas"),
            col("regiao").alias("Região"))
    .agg(count("*").alias("qtd"))
    .orderBy("qtd", ascending=False)
)

sintomas_regiao.show(truncate=False)

+-------------------------------------------------+------------+----+
|Sintomas                                         |Região      |qtd |
+-------------------------------------------------+------------+----+
|Na semana passada teve dor de cabeça?            |Nordeste    |5360|
|Na semana passada teve dor de cabeça?            |Sudeste     |4858|
|Na semana passada teve dor de garganta?          |Nordeste    |3721|
|Na semana passada teve tosse?                    |Nordeste    |3403|
|Na semana passada teve dor de cabeça?            |Sul         |3393|
|Na semana passada teve tosse?                    |Sudeste     |2960|
|Na semana passada teve dor de garganta?          |Sudeste     |2505|
|Na semana passada teve dor de cabeça?            |Norte       |2447|
|Na semana passada teve dor de cabeça?            |Centro-Oeste|2215|
|Na semana passada teve tosse?                    |Sul         |2100|
|Na semana passada teve febre?                    |Nordeste    |1922|
|Na semana passada t

In [ ]:
# Sintomas por sexo
sintomas_sexo = (
    fato
    .filter(col("descricao_resposta") == "Sim")
    .filter(col("cod_pergunta").isin(
        "B0011","B0012","B0013","B0014",
        "B0015","B0016","B0017","B0019"
    ))
    .groupBy(
        col("descricao_pergunta").alias("Sintomas"),
        col("descricao_sexo").alias("Sexo"))
    .agg(count("*").alias("qtd"))
    .orderBy("descricao_pergunta", "qtd", ascending=False)
)

sintomas_sexo.show(truncate=False)


+-------------------------------------------------+------+-----+
|Sintomas                                         |Sexo  |qtd  |
+-------------------------------------------------+------+-----+
|Na semana passada teve tosse?                    |Mulher|6205 |
|Na semana passada teve tosse?                    |Homem |5435 |
|Na semana passada teve náusea?                   |Mulher|2656 |
|Na semana passada teve náusea?                   |Homem |1548 |
|Na semana passada teve febre?                    |Mulher|3255 |
|Na semana passada teve febre?                    |Homem |2859 |
|Na semana passada teve fadiga?                   |Mulher|3876 |
|Na semana passada teve fadiga?                   |Homem |2685 |
|Na semana passada teve dor no peito?             |Mulher|3993 |
|Na semana passada teve dor no peito?             |Homem |2835 |
|Na semana passada teve dor de garganta?          |Mulher|6186 |
|Na semana passada teve dor de garganta?          |Homem |5036 |
|Na semana passada teve d

#### Sintomas por tipo de gravidade

In [ ]:
tipo_sintoma = (
    fato.filter(col("descricao_resposta") == "Sim")
    .filter(col("tipo_sintoma").isNotNull())\
    .groupBy(col("tipo_sintoma").alias("Classificação Sintoma"))\
    .agg(count("*").alias("qtd"))
    .orderBy(col("qtd").desc())
)

tipo_sintoma.show(truncate=False)

+---------------------+-----+
|Classificação Sintoma|qtd  |
+---------------------+-----+
|Sintoma geral        |35152|
|Respiratório leve    |22862|
|Respiratório grave   |13257|
+---------------------+-----+


In [ ]:
tipo_sintoma_regiao = (
    fato.filter(col("descricao_resposta") == "Sim")
    .filter(col("tipo_sintoma").isNotNull())\
    .groupBy(
        col("regiao").alias("Região"),
        col("tipo_sintoma").alias("Classificação Sintoma"))\
    .agg(count("*").alias("qtd"))
    .orderBy(
        col("Região"),
        col("qtd").desc())
)

tipo_sintoma_regiao.show(truncate=False)

+------------+---------------------+----+
|Região      |Classificação Sintoma|qtd |
+------------+---------------------+----+
|Centro-Oeste|Sintoma geral        |4492|
|Centro-Oeste|Respiratório leve    |2658|
|Centro-Oeste|Respiratório grave   |1664|
|Nordeste    |Sintoma geral        |9878|
|Nordeste    |Respiratório leve    |7124|
|Nordeste    |Respiratório grave   |3395|
|Norte       |Sintoma geral        |5400|
|Norte       |Respiratório leve    |3617|
|Norte       |Respiratório grave   |2148|
|Sudeste     |Sintoma geral        |8925|
|Sudeste     |Respiratório leve    |5465|
|Sudeste     |Respiratório grave   |3492|
|Sul         |Sintoma geral        |6457|
|Sul         |Respiratório leve    |3998|
|Sul         |Respiratório grave   |2558|
+------------+---------------------+----+


In [ ]:
#Sintoma por faixa etaria sintomas graves
tipo_sintoma_idade = (
    fato
    .filter(col("descricao_resposta") == "Sim")
    .filter(col("tipo_sintoma") == "Respiratório grave")
    .groupBy(
        col("faixa_etaria").alias("Faixa Etaria"),
        col("tipo_sintoma").alias("Classificação Sintoma")
    )
    .agg(count("*").alias("qtd"))
    .orderBy(
        col("qtd").desc()
    )
)

tipo_sintoma_idade.show(truncate=False)

+------------+---------------------+----+
|Faixa Etaria|Classificação Sintoma|qtd |
+------------+---------------------+----+
|31-40       |Respiratório grave   |2452|
|41-50       |Respiratório grave   |2168|
|21-30       |Respiratório grave   |1998|
|51-60       |Respiratório grave   |1920|
|11-20       |Respiratório grave   |1513|
|61-70       |Respiratório grave   |1229|
|0-10        |Respiratório grave   |1118|
|71+         |Respiratório grave   |859 |
+------------+---------------------+----+


### Comportamental

#### Procura por atendimento de saúde

In [ ]:
atendimento = (
    fato
    .filter(col("cod_pergunta") == "B002")
    .groupBy(col("descricao_resposta").alias("Procurou atendimento de saúde?"))
    .agg(count("*").alias("qtd"))
)

atendimento.show(truncate=False)



+------------------------------+------+
|Procurou atendimento de saúde?|qtd   |
+------------------------------+------+
|Não                           |401433|
|Sim                           |9234  |
|Ignorado                      |1368  |
+------------------------------+------+


In [ ]:
'''
pdf = atendimento.toPandas()

plt.figure(figsize=(10,4))
plt.bar(pdf["Procurou atendimento de saúde?"], pdf["qtd"])
plt.title("Procura por estabelecimento de saúde")
plt.xlabel("Resposta")
plt.ylabel("Quantidade de pessoas")

# Rótulos de dados
for i, v in enumerate(pdf["qtd"]):
    plt.text(
        i,          # posição no eixo X
        v + 1,      # acima da barra
        str(v),
        ha="center",
        va="bottom"
    )


plt.subplots_adjust(top=0.9)
plt.tight_layout()
plt.savefig("/tmp/atendimento.png")
plt.close()

salvar_no_s3(
    "/tmp/atendimento.png",
    "lab-232736165724",
    "data-output/graficos/atendimento.png"
)


'''

'\npdf = atendimento.toPandas()\n\nplt.figure(figsize=(10,4))\nplt.bar(pdf["Procurou atendimento de saúde?"], pdf["qtd"])\nplt.title("Procura por estabelecimento de saúde")\nplt.xlabel("Resposta")\nplt.ylabel("Quantidade de pessoas")\n\n# Rótulos de dados\nfor i, v in enumerate(pdf["qtd"]):\n    plt.text(\n        i,          # posição no eixo X\n        v + 1,      # acima da barra\n        str(v),\n        ha="center",\n        va="bottom"\n    )\n\n\nplt.subplots_adjust(top=0.9)\nplt.tight_layout()\nplt.savefig("/tmp/atendimento.png")\nplt.close()\n\nsalvar_no_s3(\n    "/tmp/atendimento.png",\n    "lab-232736165724",\n    "data-output/graficos/atendimento.png"\n)\n\n\n'


In [ ]:
#Procurou atendimento por Estado
atendimento_UF = (
    fato
    .filter(col("cod_pergunta") == "B002")
    .filter(col("descricao_resposta") == "Sim")
    .groupBy(
        col("nome_uf").alias("Estado"),
        col("descricao_resposta").alias("Procurou atendimento de saúde?")
    )
    .agg(count("*").alias("qtd"))
    .orderBy(
        col("qtd").desc()
    )
)

atendimento_UF.show(truncate=False)

+------------------+------------------------------+---+
|Estado            |Procurou atendimento de saúde?|qtd|
+------------------+------------------------------+---+
|Minas Gerais      |Sim                           |846|
|São Paulo         |Sim                           |776|
|Rio Grande do Sul |Sim                           |661|
|Santa Catarina    |Sim                           |651|
|Paraná            |Sim                           |530|
|Goiás             |Sim                           |483|
|Rio de Janeiro    |Sim                           |461|
|Maranhão          |Sim                           |403|
|Bahia             |Sim                           |398|
|Ceará             |Sim                           |344|
|Pará              |Sim                           |327|
|Espírito Santo    |Sim                           |325|
|Mato Grosso       |Sim                           |324|
|Amazonas          |Sim                           |322|
|Pernambuco        |Sim                         

In [ ]:
# Não Procurou atendimento
atendimento_UF = (
    fato
    .filter(col("cod_pergunta") == "B002")
    .filter(col("descricao_resposta") == "Não")
    .groupBy(
        col("nome_uf").alias("Estado"),
        col("descricao_resposta").alias("Procurou atendimento de saúde?")
    )
    .agg(count("*").alias("qtd"))
    .orderBy(
        col("qtd").desc()
    )
)

atendimento_UF.show(truncate=False)

+------------------+------------------------------+-----+
|Estado            |Procurou atendimento de saúde?|qtd  |
+------------------+------------------------------+-----+
|Minas Gerais      |Não                           |36572|
|São Paulo         |Não                           |33776|
|Rio de Janeiro    |Não                           |29915|
|Santa Catarina    |Não                           |23992|
|Rio Grande do Sul |Não                           |23836|
|Maranhão          |Não                           |21675|
|Paraná            |Não                           |20853|
|Bahia             |Não                           |18444|
|Pernambuco        |Não                           |17865|
|Ceará             |Não                           |17115|
|Espírito Santo    |Não                           |16466|
|Goiás             |Não                           |15156|
|Pará              |Não                           |12560|
|Alagoas           |Não                           |11912|
|Mato Grosso  

#### Providências tomadas para recuperação

In [ ]:
providencias = (
    fato
    .filter(col("cod_pergunta").isin("B0032","B0033","B0034","B0037"))
    .filter(col("descricao_resposta") == "Sim")
    .groupBy("descricao_pergunta")
    .agg(count("*").alias("qtd"))
    .orderBy("qtd", ascending=False)
)

providencias.show(truncate=False)


+---------------------------------------------------------------------------------------------------+-----+
|descricao_pergunta                                                                                 |qtd  |
+---------------------------------------------------------------------------------------------------+-----+
|Providência tomada  para recuperar dos sintomas foi comprar e/ou tomar  remédio por conta própria  |17969|
|Providência tomada para recuperar dos sintomas foi comprar e/ou tomar remédio por orientação médica|10031|
|Providência tomada para recuperar dos sintomas foi ligar para algum profissional de saúde          |2107 |
|Providência tomada para recuperar dos sintomas foi outra                                           |1558 |
+---------------------------------------------------------------------------------------------------+-----+


In [ ]:
'''
pdf = providencias.toPandas()

plt.figure(figsize=(20,6))

plt.barh(pdf["descricao_pergunta"], pdf["qtd"])

plt.title(
    "Providências tomadas para recuperação dos sintomas",
    pad=20,
    loc="center"
)

plt.xlabel("Quantidade de pessoas")
plt.ylabel("Providência")
plt.gca().invert_yaxis()

# Rótulos de dados
for i, v in enumerate(pdf["qtd"]):
    plt.text(v + 1, i, str(v), va="center")

plt.subplots_adjust(top=0.9)
plt.tight_layout()

plt.savefig("/tmp/providencias.png")
plt.close()


salvar_no_s3(
    "/tmp/providencias.png",
    "lab-232736165724",
    "data-output/graficos/providencias.png"
)
'''

'\npdf = providencias.toPandas()\n\nplt.figure(figsize=(20,6))\n\nplt.barh(pdf["descricao_pergunta"], pdf["qtd"])\n\nplt.title(\n    "Providências tomadas para recuperação dos sintomas",\n    pad=20,\n    loc="center"\n)\n\nplt.xlabel("Quantidade de pessoas")\nplt.ylabel("Providência")\nplt.gca().invert_yaxis()\n\n# Rótulos de dados\nfor i, v in enumerate(pdf["qtd"]):\n    plt.text(v + 1, i, str(v), va="center")\n\nplt.subplots_adjust(top=0.9)\nplt.tight_layout()\n\nplt.savefig("/tmp/providencias.png")\nplt.close()\n\n\nsalvar_no_s3(\n    "/tmp/providencias.png",\n    "lab-232736165724",\n    "data-output/graficos/providencias.png"\n)\n'


In [ ]:
# Providência tomada por sexo
providencias_sexo = (
    fato
    .filter(col("cod_pergunta").isin("B0032","B0033","B0034","B0037"))
    .filter(col("descricao_resposta") == "Sim")
    .groupBy(
        col("descricao_pergunta").alias("Providencia"),
        col("descricao_sexo").alias("Sexo"))
    .agg(count("*").alias("qtd"))
    .orderBy(
        col("qtd").desc(),
        col("Providencia"))
)

providencias_sexo.show(truncate=False)

+---------------------------------------------------------------------------------------------------+------+-----+
|Providencia                                                                                        |Sexo  |qtd  |
+---------------------------------------------------------------------------------------------------+------+-----+
|Providência tomada  para recuperar dos sintomas foi comprar e/ou tomar  remédio por conta própria  |Mulher|10516|
|Providência tomada  para recuperar dos sintomas foi comprar e/ou tomar  remédio por conta própria  |Homem |7453 |
|Providência tomada para recuperar dos sintomas foi comprar e/ou tomar remédio por orientação médica|Mulher|6074 |
|Providência tomada para recuperar dos sintomas foi comprar e/ou tomar remédio por orientação médica|Homem |3957 |
|Providência tomada para recuperar dos sintomas foi ligar para algum profissional de saúde          |Mulher|1226 |
|Providência tomada para recuperar dos sintomas foi outra                       

#### Internação hospitalar

In [ ]:
internacao = (
    fato
    .filter(col("cod_pergunta") == "B005")
    .groupBy(col("descricao_resposta").alias("Houve Internação hospitalar?"))
    .agg(count("*").alias("qtd"))
)

internacao.show(truncate=False)


+----------------------------+----+
|Houve Internação hospitalar?|qtd |
+----------------------------+----+
|Não foi atendido            |35  |
|Não                         |6924|
|Sim                         |765 |
|Ignorado                    |36  |
+----------------------------+----+


In [ ]:
internacao_sexo = (
    fato
    .filter(col("cod_pergunta") == "B005")
    .filter(col("descricao_resposta") == "Sim")
    .groupBy(
        col("descricao_resposta").alias("Houve Internação hospitalar?"),
        col("descricao_sexo").alias("Sexo"))
    .agg(count("*").alias("qtd"))
)

internacao_sexo.show(truncate=False)

+----------------------------+------+---+
|Houve Internação hospitalar?|Sexo  |qtd|
+----------------------------+------+---+
|Sim                         |Mulher|414|
|Sim                         |Homem |351|
+----------------------------+------+---+


In [ ]:
# Internação por Estado
internacao_uf = (
    fato
    .filter(col("cod_pergunta") == "B005")
    .filter(col("descricao_resposta") == "Sim")
    .groupBy(
        col("nome_uf").alias("Estado"),
        col("descricao_resposta").alias("Houve Internação hospitalar?")
        )
    .agg(count("*").alias("qtd"))
    .orderBy(col("qtd").desc())
)

internacao_uf.show(truncate=False)

+------------------+----------------------------+---+
|Estado            |Houve Internação hospitalar?|qtd|
+------------------+----------------------------+---+
|Minas Gerais      |Sim                         |88 |
|Rio Grande do Sul |Sim                         |80 |
|São Paulo         |Sim                         |69 |
|Goiás             |Sim                         |54 |
|Paraná            |Sim                         |51 |
|Rio de Janeiro    |Sim                         |50 |
|Santa Catarina    |Sim                         |46 |
|Ceará             |Sim                         |31 |
|Espírito Santo    |Sim                         |29 |
|Pernambuco        |Sim                         |28 |
|Mato Grosso       |Sim                         |28 |
|Bahia             |Sim                         |27 |
|Paraíba           |Sim                         |24 |
|Mato Grosso do Sul|Sim                         |22 |
|Amazonas          |Sim                         |17 |
|Distrito Federal  |Sim     

### Características econômicas da sociedade

#### Impacto no trabalho

In [ ]:
trabalho = (
    fato
    .filter(col("cod_pergunta") == "C001")
    .groupBy(col("descricao_resposta").alias("Trabalhou ou fez bico?"))
    .agg(count("*").alias("qtd"))
)

trabalho.show(truncate=False)


+----------------------+------+
|Trabalhou ou fez bico?|qtd   |
+----------------------+------+
|Não                   |720669|
|Sim                   |293278|
+----------------------+------+


In [ ]:
afastamento = (
    fato
    .filter(col("cod_pergunta") == "C002")
    .groupBy(col("descricao_resposta").alias("Houve Afastamento do Trabalho?"))
    .agg(count("*").alias("qtd"))
)

afastamento.show(truncate=False)


+------------------------------+------+
|Houve Afastamento do Trabalho?|qtd   |
+------------------------------+------+
|Não                           |698947|
|Sim                           |33384 |
+------------------------------+------+


#### Impacto econômico

In [ ]:
#Solicitação de empréstimo
emprestimo = (
    fato
    .filter(col("cod_pergunta") == "E001")
    .groupBy(col("descricao_resposta").alias("Solicitou emprestimo?"))
    .agg(count("*").alias("qtd"))
)

emprestimo.show(truncate=False)


+--------------------------------------+------+
|Solicitou emprestimo?                 |qtd   |
+--------------------------------------+------+
|Não solicitou                         |704670|
|Sim, e pelo menos um morador conseguiu|53969 |
|Sim, mas nenhum morador conseguiu     |9120  |
+--------------------------------------+------+


In [ ]:
#Gráfico
'''
pdf = emprestimo.toPandas()

plt.figure(figsize=(10,4))

plt.barh(pdf["Solicitou emprestimo?"], pdf["qtd"])

plt.title("Solicitação de empréstimo durante a pandemia")
plt.xlabel("Quantidade de domicílios")
plt.ylabel("Resposta")

# Rótulos de dados
for i, v in enumerate(pdf["qtd"]):
    plt.text(
        v + 1,
        i,
        str(v),
        va="center"
    )

plt.subplots_adjust(left=0.35, top=0.9)

plt.savefig("/tmp/emprestimo.png")
plt.close()

salvar_no_s3(
    "/tmp/emprestimo.png",
    "lab-232736165724",
    "data-output/graficos/emprestimo.png"
)
'''

'\npdf = emprestimo.toPandas()\n\nplt.figure(figsize=(10,4))\n\nplt.barh(pdf["Solicitou emprestimo?"], pdf["qtd"])\n\nplt.title("Solicitação de empréstimo durante a pandemia")\nplt.xlabel("Quantidade de domicílios")\nplt.ylabel("Resposta")\n\n# Rótulos de dados\nfor i, v in enumerate(pdf["qtd"]):\n    plt.text(\n        v + 1,   \n        i,       \n        str(v),\n        va="center"\n    )\n\nplt.subplots_adjust(left=0.35, top=0.9)\n\nplt.savefig("/tmp/emprestimo.png")\nplt.close()\n\nsalvar_no_s3(\n    "/tmp/emprestimo.png",\n    "lab-232736165724",\n    "data-output/graficos/emprestimo.png"\n)\n'


## AGREGAÇÕES PARA CONSUMO ANALÍTICO (CSV)

Objetivo:
 Reduzir a granularidade dos dados mantendo o máximo
 de informação analítica possível, viabilizando o
 consumo no Power BI com melhor performance.

 As agregações foram organizadas por tema:
 - Sintomas clínicos
 - Comportamento da população
 - Impacto econômico

In [ ]:
# Nomes das colunas
fato.printSchema()
fato.show(5, truncate=False)


root
 |-- capital: integer (nullable = true)
 |-- uf: integer (nullable = true)
 |-- cod_pergunta: string (nullable = true)
 |-- cod_resposta: integer (nullable = true)
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- idade: integer (nullable = true)
 |-- sexo: integer (nullable = true)
 |-- raca: integer (nullable = true)
 |-- escolaridade: integer (nullable = true)
 |-- descricao_pergunta: string (nullable = true)
 |-- descricao_resposta: string (nullable = true)
 |-- nome_uf: string (nullable = true)
 |-- descricao_capital: string (nullable = true)
 |-- cod_sexo: integer (nullable = true)
 |-- descricao_sexo: string (nullable = true)
 |-- cod_raca: integer (nullable = true)
 |-- descricao_raca: string (nullable = true)
 |-- cod_escolaridade: integer (nullable = true)
 |-- descricao_escolaridade: string (nullable = true)
 |-- regiao: string (nullable = false)
 |-- faixa_etaria: string (nullable = false)
 |-- tipo_sintoma: string (nullable = true)

+-------

In [ ]:
# Agregação dos sintomas para criar um csv que será utilizado no Power BI

sintomas_pbi = (
    fato
    .filter(col("descricao_resposta") == "Sim")
    .groupBy(
        "ano",
        "mes",
        "regiao",
        "uf",
        "nome_uf",
        "descricao_sexo",
        "faixa_etaria",
        "tipo_sintoma",
        "descricao_pergunta"
    )
    .count()
    .withColumnRenamed("count", "qtd_registros")
)

sintomas_pbi.show(truncate=False)

+----+---+------------+---+-----------------+--------------+------------+-------------+-------------------------------------+-------------+
|ano |mes|regiao      |uf |nome_uf          |descricao_sexo|faixa_etaria|tipo_sintoma |descricao_pergunta                   |qtd_registros|
+----+---+------------+---+-----------------+--------------+------------+-------------+-------------------------------------+-------------+
|2020|9  |Sudeste     |33 |Rio de Janeiro   |Homem         |31-40       |Sintoma geral|Na semana passada teve dor de cabeça?|22           |
|2020|9  |Sudeste     |33 |Rio de Janeiro   |Mulher        |51-60       |Sintoma geral|Na semana passada teve dor de cabeça?|62           |
|2020|9  |Sudeste     |33 |Rio de Janeiro   |Homem         |0-10        |Sintoma geral|Na semana passada teve fadiga?       |1            |
|2020|9  |Sudeste     |35 |São Paulo        |Homem         |51-60       |Sintoma geral|Na semana passada teve fadiga?       |15           |
|2020|9  |Sul       

In [ ]:
# Agregação do comportamental para criar um csv que será utilizado no Power BI

perguntas_comportamentais = [
    "B002",
    "B005",
    "B0032",
    "B0033",
    "B0034",
    "B0037"
]

comportamento_pbi = (
    fato
    .filter(col("cod_pergunta").isin(perguntas_comportamentais))
    .groupBy(
        "ano",
        "mes",
        "regiao",
        "descricao_sexo",
        "faixa_etaria",
        "descricao_pergunta",
        "descricao_resposta"
    )
    .count()
    .withColumnRenamed("count", "qtd_registros")
)
comportamento_pbi.show(truncate=False)

+----+---+------------+--------------+------------+---------------------------------------------------------------------------------------------------+------------------+-------------+
|ano |mes|regiao      |descricao_sexo|faixa_etaria|descricao_pergunta                                                                                 |descricao_resposta|qtd_registros|
+----+---+------------+--------------+------------+---------------------------------------------------------------------------------------------------+------------------+-------------+
|2020|9  |Sul         |Mulher        |51-60       |Providência tomada para recuperar dos sintomas foi comprar e/ou tomar remédio por orientação médica|Não               |196          |
|2020|9  |Sul         |Mulher        |61-70       |Providência tomada para recuperar dos sintomas foi comprar e/ou tomar remédio por orientação médica|Não               |165          |
|2020|9  |Sul         |Homem         |31-40       |Providência tomada para 

In [ ]:
# Agregação relacionada a economia para criar um csv que será utilizado no Power BI
perguntas_economicas = [
    "C001",
    "C002",
    "C003",
    "C005",
    "C006",
    "E001"
]

economico_pbi = (
    fato
    .filter(col("cod_pergunta").isin(perguntas_economicas))
    .groupBy(
        "ano",
        "mes",
        "regiao",
        "descricao_sexo",
        "faixa_etaria",
        "descricao_pergunta",
        "descricao_resposta"
    )
    .count()
    .withColumnRenamed("count", "qtd_registros")
)

economico_pbi.show(truncate=False)

+----+---+------------+--------------+------------+-----------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+-------------+
|ano |mes|regiao      |descricao_sexo|faixa_etaria|descricao_pergunta                                   |descricao_resposta                                                                                                                   |qtd_registros|
+----+---+------------+--------------+------------+-----------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+-------------+
|2020|9  |Sudeste     |Mulher        |21-30       |Há quanto tempo está afastado desse trabalho?        |De 1 mês a menos de 1 ano                                                                                                            

### Exportar para CSV


In [ ]:
# Exportar sintomas pbi
(
    sintomas_pbi
    .coalesce(1)
    .write
    .mode("overwrite")
    .option("header", "true")
    .csv("s3://lab-232736165724/data-output/pbi/sintomas_pbi/")
)


In [ ]:
# Exportar comportamento pbi
(
    comportamento_pbi
    .coalesce(1)
    .write
    .mode("overwrite")
    .option("header", "true")
    .csv("s3://lab-232736165724/data-output/pbi/comportamento_pbi/")
)

In [ ]:
# Exportar economico pbi
(
    economico_pbi
    .coalesce(1)
    .write
    .mode("overwrite")
    .option("header", "true")
    .csv("s3://lab-232736165724/data-output/pbi/economico_pbi/")
)

# CONCLUSÃO DO NOTEBOOK

 Neste notebook foi realizado:
 - Exploração e validação dos dados da PNAD COVID-19
 - Criação de variáveis derivadas para análises demográficas
 - Agregações temáticas para saúde, comportamento e economia
 - Preparação dos dados para consumo analítico no Power BI
